# Assignment 3 - Identification of railway track events
### Industrial AI and eMainteance – Part I, D7015B

<p style="text-align: right;"><em>by Martin Olin</em></p>


### Problem description

One way to detect issues with the railway track is to measure the interactions between the wheels and the railway track with acceleration sensors that pick up vibrations and by analyzing the measured frequencies and wave patterns one can deduce defects. In this assignment such measurements are provided and analyzied for classification by a linear SVM. 

### Overview
To accomplish this, the data is first loaded from three different sources and combined into a single dataset. Irrelevant data is discarded, and the remaining data is binary classified as either "good" or "defect." Next, the dataset is split into training and test sets using an 80/20 ratio, and 5-fold cross-validation is performed to evaluate model stability. Finally, two filter methods and two wrapper methods are demonstrated to show how they can be used to preprocess the data and reduce dimensionality.

### Method

First a few libraries were loaded as shown by the code below:

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
np.set_printoptions(legacy='1.25') #nicer print
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Next the data sources were loaded, and as demonstrated these does contain different number of features: 

In [2]:
df_trail1 = pd.read_csv('Trail1_extracted_features_acceleration_m1ai1-1.csv')
df_trail2 = pd.read_csv('Trail2_extracted_features_acceleration_m1ai1.csv')
df_trail3 = pd.read_csv('Trail3_extracted_features_acceleration_m2ai0.csv')

dfs=[df_trail1, df_trail2, df_trail3]

print(df_trail1.shape)
print(df_trail2.shape)
print(df_trail3.shape)

(52, 19)
(49, 19)
(49, 22)


The names of these were as follows:

In [3]:
for data in dfs:
    print(data.columns)

Index(['mean', 'std', 'max', 'min', 'range', 'skewness', 'kurtosis', 'rms',
       'crest_factor', 'variance', 'zero_crossings', 'dominant_freq',
       'spectral_energy', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_flatness', 'start_time', 'event', 'axle'],
      dtype='object')
Index(['mean', 'std', 'max', 'min', 'range', 'skewness', 'kurtosis', 'rms',
       'crest_factor', 'variance', 'zero_crossings', 'dominant_freq',
       'spectral_energy', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_flatness', 'start_time', 'event', 'axle'],
      dtype='object')
Index(['mean', 'std', 'max', 'min', 'range', 'skewness', 'kurtosis', 'rms',
       'crest_factor', 'variance', 'zero_crossings', 'dominant_freq',
       'spectral_energy', 'spectral_centroid', 'spectral_bandwidth',
       'spectral_flatness', 'start_time', 'event', 'axle', 'cluster', 'tsne_1',
       'tsne_2'],
      dtype='object')


The data was not normalized to start with, here showing 4 data points:

In [4]:
df_trail3.sample(n=4, random_state=24)

,mean,std,max,min,range,skewness,kurtosis,rms,crest_factor,variance,...,spectral_energy,spectral_centroid,spectral_bandwidth,spectral_flatness,start_time,event,axle,cluster,tsne_1,tsne_2
8,-0.000039,0.013216,0.040186,-0.057198,0.097384,-0.227469,0.167842,0.013216,3.040798,0.000175,...,0.000007,302.214869,384.529489,0.004821,9.901992,normal,normal,0,-2.349625,5.166960
5,0.000015,0.078127,0.582892,-0.804829,1.387721,-0.781602,21.367413,0.078127,7.460853,0.006104,...,0.000246,313.137674,196.101692,0.000318,8.238000,joint X,A2,1,1.899679,5.718000
30,-0.000046,0.031775,0.282569,-0.258469,0.541038,0.056260,13.399656,0.031775,8.892871,0.001010,...,0.000040,268.376952,205.494690,0.000959,31.475000,crossing,A1,0,-0.467289,5.115012
29,-0.000008,0.017330,0.098209,-0.102123,0.200331,-0.039548,3.875768,0.017330,5.667109,0.000300,...,0.000012,268.200128,308.119171,0.002903,30.929000,squat H,A1,0,-1.626651,4.825243


The third data set had 3 extra columns in the end; cluster, tsne_1 and tsne_2. In order to combine all dataset these data columns were dropped by the code below and the data sets were combined into one common dataset (df_combined): 

In [5]:
dfs[2]=dfs[2].drop(['cluster'	,'tsne_1','tsne_2'],axis=1)
df_combined= pd.concat(dfs,axis=0)
print(df_combined.shape)
df_combined.sample(n=2, random_state=24)

(150, 19)


,mean,std,max,min,range,skewness,kurtosis,rms,crest_factor,variance,zero_crossings,dominant_freq,spectral_energy,spectral_centroid,spectral_bandwidth,spectral_flatness,start_time,event,axle
6,-0.000018,0.078133,0.582892,-0.804829,1.387721,-0.780089,21.357968,0.078133,7.460227,0.006105,931,325.0,0.000246,312.947675,193.284416,0.000317,8.498000,squat B,A1
13,0.000065,0.013004,0.042284,-0.050154,0.092438,-0.016909,0.600969,0.013005,3.251452,0.000169,97,300.0,0.000007,392.130148,433.008841,0.004329,13.442969,normal,normal


The Assignment also specicied that we should drop axles and start_time. We weere also to change the events into 0 for normal and 1 otherwise: 

In [6]:
df_combined=df_combined.drop(['start_time','axle'],axis=1)
df_combined['event']= df_combined['event'].apply(lambda x: 0 if 'normal' in str(x) else 1)
df_combined.sample(n=2, random_state=24)

,mean,std,max,min,range,skewness,kurtosis,rms,crest_factor,variance,zero_crossings,dominant_freq,spectral_energy,spectral_centroid,spectral_bandwidth,spectral_flatness,event
6,-0.000018,0.078133,0.582892,-0.804829,1.387721,-0.780089,21.357968,0.078133,7.460227,0.006105,931,325.0,0.000246,312.947675,193.284416,0.000317,1
13,0.000065,0.013004,0.042284,-0.050154,0.092438,-0.016909,0.600969,0.013005,3.251452,0.000169,97,300.0,0.000007,392.130148,433.008841,0.004329,0


Next to normalize all data the following code was used:

In [7]:

scaleing=MinMaxScaler()
df_preprocessed_grade_3=pd.DataFrame(scaleing.fit_transform(df_combined), columns=df_combined.columns)

In order to split the data into a test and training part 80/20, train_test_split was used:

In [9]:
y=df_preprocessed_grade_3['event']
x=df_preprocessed_grade_3.drop(['event'],axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2, random_state=25)

#### Grade 4

Cross-valdiation was performed by the code below:

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:

def CrossVal(X_train,Y_train,k=5):
    kfoldsize=round(len(X_train)/k) #need to be deviceable with k, this is the case here 120/5 so no checks are implemented.
    accuracies= []
    for i in range(k):
        X_test_cross=[]
        X_train_cross=[]
        Y_test_cross=[]
        Y_train_cross=[]
        for fold in range(k):
            if fold==i:
                X_test_cross=X_train[kfoldsize*i:kfoldsize*(i+1)]
                Y_test_cross=Y_train[kfoldsize*i:kfoldsize*(i+1)]
            else:
                X_train_cross.append(X_train[kfoldsize*fold:kfoldsize*(fold+1)])
                Y_train_cross.append(Y_train[kfoldsize*fold:kfoldsize*(fold+1)])
        X_train_cross= pd.concat(X_train_cross,axis=0)
        Y_train_cross= pd.concat(Y_train_cross,axis=0)

        model=SVC()
        model.fit(X_train_cross,Y_train_cross)
        y_pred= model.predict(X_test_cross)
        accuracies.append(accuracy_score(Y_test_cross,y_pred))
    return accuracies
accuracies=CrossVal(X_train,Y_train,5)

for acc in range(len(accuracies)):
    print("fold nr " + str(acc) + " got acc:" + str(accuracies[acc]))
print("model accuracy on average = " + str(np.mean(accuracies)))

fold nr 0 got acc:1.0
fold nr 1 got acc:0.9583333333333334
fold nr 2 got acc:0.875
fold nr 3 got acc:1.0
fold nr 4 got acc:0.9166666666666666
model accuracy on average = 0.95


The average cross-validation result could be compared to what happens when all the training data is used for training the model:

In [29]:
model=SVC()
model.fit(X_train,Y_train)
y_pred= model.predict(X_test)
acc=accuracy_score(Y_test,y_pred)
print("the accuracy from using all the training data was: " + str(acc))

the accuracy from using all the training data was: 1.0


Getting a perfect result like 100% can be problematic, still the cross-validation also yeilded high results so it is a good idea to train with all the available training data.

#### Grade 5 
Four feature selection algorithms were implemented, these are important when working with very large datasets but not really needed in this case.

The Code below demonstrates a filter method, Pearson Correlation, a low correlation means that we can remove the feautre:

In [13]:
corr_matrix=df_combined.corr(method='pearson')
corr_columns=abs(corr_matrix['event']).sort_values(ascending=True)
features= corr_columns.index.tolist()
print(corr_columns)

dominant_freq         0.013005
mean                  0.058047
zero_crossings        0.121471
skewness              0.157527
spectral_bandwidth    0.351987
spectral_centroid     0.377106
spectral_flatness     0.379142
spectral_energy       0.402178
variance              0.405861
rms                   0.490274
std                   0.490287
max                   0.508304
range                 0.525619
min                   0.531723
kurtosis              0.624201
crest_factor          0.769177
event                 1.000000
Name: event, dtype: float64


For a linear kernel we can then remove the feature with the least correlation, here showing the average accuracy from five folded cross validation and the top correlated features:

In [14]:
X_train_pearson=X_train
features=features[:-1] #remove event itself
nrOfFeatures=len(features)
for feature in features:
    if nrOfFeatures>1:
        X_train_pearson=X_train_pearson.drop(feature,axis=1)
        nrOfFeatures=nrOfFeatures-1
        acc=CrossVal(X_train_pearson,Y_train)
        print(str(nrOfFeatures) + " top features "+ "correlations above absolut values of " + str(abs(corr_columns[feature])) +" give an mean accuracy = " + str(np.mean(acc)) + " k-fold accuracies = "+ str(acc) ) 
      

15 top features correlations above absolut values of 0.01300475519112022 give an mean accuracy = 0.9583333333333334 k-fold accuracies = [1.0, 0.9583333333333334, 0.875, 1.0, 0.9583333333333334]
14 top features correlations above absolut values of 0.05804740976371208 give an mean accuracy = 0.9583333333333334 k-fold accuracies = [1.0, 0.9583333333333334, 0.875, 1.0, 0.9583333333333334]
13 top features correlations above absolut values of 0.12147061847455003 give an mean accuracy = 0.95 k-fold accuracies = [1.0, 0.9583333333333334, 0.875, 0.9583333333333334, 0.9583333333333334]
12 top features correlations above absolut values of 0.157526602392071 give an mean accuracy = 0.95 k-fold accuracies = [1.0, 0.9583333333333334, 0.875, 0.9583333333333334, 0.9583333333333334]
11 top features correlations above absolut values of 0.35198744069748444 give an mean accuracy = 0.95 k-fold accuracies = [1.0, 0.9583333333333334, 0.875, 0.9583333333333334, 0.9583333333333334]
10 top features correlations 

The code above showed that only the top feautre (crest_factor) was really needed to train the SVM.

Next, a wrapper method was implemented: forward feature selection. All features were tested individually, and only the best-performing one was retained. Then a brute-force approach was applied by combining the best feature with each of the remaining ones. The most effective combinations were preserved, and this process was repeated iteratively.

In [15]:
features= corr_columns.index.tolist()[:-1] 
NrOfFeatures=len(features)
def BackwardFeatureSelection(X,Y,featuresWanted=1):
    testFeaturesRemoved=[]
    nrOfFeatures=len(X.columns)
    while nrOfFeatures>featuresWanted:
        scores=[]
        testfeautures=[feat for feat in X.columns if feat not in testFeaturesRemoved]
        for feature in testfeautures:
            acc=CrossVal(X[[feat for feat in X.columns if (feat!=feature and feat not in testFeaturesRemoved) ]],Y_train)
            scores.append(np.mean(acc))
        lowestScoreIndex=scores.index(min(scores))
        testFeaturesRemoved.append(testfeautures[lowestScoreIndex])
        print("scores:")
        print(scores)
        print("removing feature "+ str(testfeautures[lowestScoreIndex]) + " # worst score: " + str( min(scores))  +" features left: " + str(len(testfeautures)-1))
        nrOfFeatures=nrOfFeatures-1
    print("According to back feature selection the best feature was:")
    print([feat for feat in X.columns if feat not in testFeaturesRemoved])    
    #return X[[feat for feat in X.columns if feat not in testFeaturesRemoved]]
BackwardFeatureSelection(X_train,Y_train,1)


        
                

scores:
[0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.9583333333333334, 0.95, 0.8916666666666666, 0.95, 0.9416666666666668, 0.9583333333333334, 0.95, 0.95, 0.95, 0.9416666666666668]
removing feature crest_factor # worst score: 0.8916666666666666 features left: 15
scores:
[0.8916666666666666, 0.9083333333333332, 0.9, 0.925, 0.9083333333333332, 0.9, 0.8583333333333332, 0.9083333333333332, 0.9, 0.85, 0.8916666666666666, 0.9, 0.8916666666666666, 0.9, 0.8583333333333334]
removing feature zero_crossings # worst score: 0.85 features left: 14
scores:
[0.8583333333333334, 0.85, 0.8333333333333333, 0.8666666666666666, 0.8333333333333333, 0.85, 0.7333333333333334, 0.85, 0.85, 0.8333333333333334, 0.85, 0.8583333333333334, 0.85, 0.8333333333333334]
removing feature kurtosis # worst score: 0.7333333333333334 features left: 13
scores:
[0.7333333333333333, 0.7250000000000001, 0.7250000000000001, 0.725, 0.725, 0.7416666666666667, 0.7250000000000001, 0.7333333333333334, 0.7333333333333333, 0.7333333333333334, 

One problem with the backwardselection method here is when removing a feature, several features gives similar scores, hence just the first one is removed in those cases. This is why crest_factor is removed at the first step, while from the pearson correlation we know this feature was very good to keep. 


In [ ]:
features= corr_columns.index.tolist()[:-1] 
NrOfFeatures=len(features)
def ForwardFeatureSelection(X,Y,featuresWanted=1):
    testFeaturesSelected=[]
    nrOfFeatures=0

    if len(X.columns)<featuresWanted:
        print("too many features, cant take more than " + str(len(X.columns)) + " features")
        featuresWanted=len(X.columns)

    while nrOfFeatures<featuresWanted:
        scores=[]
        testfeautures=[feat for feat in X.columns if feat not in testFeaturesSelected]
        for feature in testfeautures:
            acc=CrossVal(X[[feat for feat in X.columns if (feat in testFeaturesSelected) or feat ]],Y_train)
            scores.append(np.mean(acc))
        highestScoreIndex=scores.index(max(scores))
        print("scores:")
        print(scores)
        testFeaturesSelected.append(testfeautures[highestScoreIndex])
        print("Adding feature "+ str(testfeautures[highestScoreIndex]) + " # best score: " + str( max(scores))  +" features selected: " + str(len(testFeaturesSelected)))
        nrOfFeatures=nrOfFeatures+1
    print("According to forward feature selection the best features was:")
    print([feat for feat in X.columns if feat in testFeaturesSelected])
 
    #return X[[feat for feat in X.columns if feat not in testFeaturesRemoved]]
ForwardFeatureSelection(X_train,Y_train,3)


scores:
[0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
Adding feature mean # best score: 0.95 features selected: 1
scores:
[0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
Adding feature std # best score: 0.95 features selected: 2
scores:
[0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
Adding feature max # best score: 0.95 features selected: 3
According to back feature selection the best features was:
['mean', 'std', 'max']


This has the same problem as BackFeatureSelection, each feature does too little difference.

Another feature selection algothim that we can use is relief

In [31]:
from skrebate import ReliefF

relief = ReliefF(n_features_to_select=5, n_neighbors=100)
X_new = relief.fit_transform(X_train.values, Y_train.values)
relief_selected_feature_indices = relief.top_features_[:relief.n_features_to_select]

relief_selected_feature_names = X_train.columns[relief_selected_feature_indices]
print(relief_selected_feature_names)  # Feature weights

Index(['crest_factor', 'kurtosis', 'min', 'spectral_bandwidth', 'range'], dtype='object')


Using the features selected by ReliefF we get the following accuracy:


In [34]:
model=SVC()
model.fit(X_train[relief_selected_feature_names],Y_train)
y_pred= model.predict(X_test[relief_selected_feature_names])
acc=accuracy_score(Y_test,y_pred)
print("the accuracy from using all the training data but only the top 3 features from ReliefF was: " + str(acc))

the accuracy from using all the training data but only the top 3 features from ReliefF was: 0.9666666666666667


### Observations and reflections
Cross-validation can help detect overfitting; the SVM used did not show any pronounced signs of it.

Backward and Forward selection algothims does not work too well when the difference of contribution from different features are not so pronounced. ReleifF can sometimes be considered to be very computationally intensive as it has to look for close neigbours of the data points themselves but this is no issue at all with the data amount for this assignment. The pearson correlation is well suited for a linear classificer, a single feature, crest_factor gave an accuracy of 96%.